In [1]:
from dnallm import load_config, load_model_and_tokenizer, DNADataset, DNATrainer

In [2]:
# Load the config file
configs = load_config("./ner_task_config.yaml")

In [3]:
# Load the model and tokenizer
model_name = "zhangtaolab/plant-nucleotide-transformer-BPE"
# from Hugging Face
# model, tokenizer = load_model_and_tokenizer(model_name, task_config=configs['task'], source="huggingface")
# from ModelScope
model, tokenizer = load_model_and_tokenizer(model_name, task_config=configs['task'], source="modelscope")

14:37:38 - dnallm.utils.support - INFO - Model files are stored in /Users/forrest/.cache/modelscope/hub/models/zhangtaolab/plant-nucleotide-transformer-BPE


Some weights of EsmForTokenClassification were not initialized from the model checkpoint at /Users/forrest/.cache/modelscope/hub/models/zhangtaolab/plant-nucleotide-transformer-BPE and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
# Load the datasets
datasets = DNADataset.load_local_data("./rice_gene_ner_BPE.pkl", seq_col="sequence", label_col="labels", tokenizer=tokenizer, max_length=1024)

# Encode the sequences with given task's data collator
datasets.encode_sequences(task=configs['task'].task_type, remove_unused_columns=True)

# Split the dataset into train, test, and validation sets
datasets.split_data()

Format labels:   0%|          | 0/2000 [00:00<?, ? examples/s]

Encoding inputs:   0%|          | 0/2000 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (2067 > 2048). Running this sequence through the model will result in indexing errors


In [5]:
# Initialize the trainer
trainer = DNATrainer(
    model=model,
    config=configs,
    datasets=datasets
)

In [6]:
# Start training
metrics = trainer.train()
print(metrics)

Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
500,0.572600,0.312247,0.901479,0.611558,0.566396,0.588111
1000,0.164400,0.187498,0.945075,0.767223,0.716802,0.741156


{'train_runtime': 3382.1381, 'train_samples_per_second': 1.241, 'train_steps_per_second': 0.31, 'total_flos': 2381880829722624.0, 'train_loss': 0.35612290246146067, 'epoch': 3.0}


In [7]:
# Do prediction on the test set
trainer.infer()

PredictionOutput(predictions=array([[[ 2.323482  , -1.5099787 , -0.8836714 , ..., -0.927853  ,
         -1.4103745 ,  0.04891092],
        [ 5.6943636 , -1.5035043 , -0.9116931 , ..., -1.6362175 ,
         -2.0631716 , -0.94190305],
        [ 5.8905754 , -0.913214  , -1.2376132 , ..., -1.6290904 ,
         -1.4768428 , -1.1819894 ],
        ...,
        [-0.30754197, -1.0372043 ,  0.7798816 , ...,  0.88006437,
         -1.8100628 ,  1.5709177 ],
        [-0.9096149 , -1.5369973 ,  1.5390666 , ...,  2.3269818 ,
         -1.854856  ,  1.2788448 ],
        [-1.1789548 , -1.90242   ,  1.672185  , ...,  3.0487711 ,
         -2.1100445 ,  1.5636654 ]],

       [[ 2.1773806 , -1.733074  , -0.8279599 , ..., -1.1372741 ,
         -1.5287644 ,  0.13668065],
        [ 5.831868  , -0.84064394, -1.1946642 , ..., -1.2554147 ,
         -1.5157167 , -1.4430664 ],
        [ 6.172637  , -1.1160768 , -0.8910407 , ..., -1.0433854 ,
         -2.0278275 , -1.6230795 ],
        ...,
        [ 0.20084313, -1.